In [1]:
import dask.dataframe as dd
from py2neo import Graph
from tqdm.auto import tqdm
import time
from searchapp.models import Entity
from tqdm.dask import TqdmCallback

In [2]:
ddf = dd.read_csv('data/conceptnet-assertions-5.7.0.csv', header=None, sep='\t', names=['uri','relation','node_start','node_end','info'])
ddf = ddf[ddf['node_start'].str.startswith('/c/en') & ddf['node_end'].str.startswith('/c/en')]
ddf

,uri,relation,node_start,node_end,info
npartitions=158,,,,,
,object,object,object,object,object
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [3]:
cb = TqdmCallback(desc="global")
cb.register()
# names = list(dd.concat([ddf['node_start'], ddf['node_end']]).unique().compute())
names = list(ddf['node_start'].unique().compute())

global:   0%|          | 0/182 [00:00<?, ?it/s]

In [4]:
len(names)

1825430

In [5]:
k = 100000
for i in tqdm(range(0, len(names), k)):
    _data = names[i:i+k]
    data = [
        Entity(
            id=elm[:300],
            name=elm.replace('/c/en/','')[:300], 
            attribute={
                'dataset': "ConceptNet"
            }
        ) for elm in _data
    ]
    Entity.objects.bulk_create(data)

  0%|          | 0/15 [00:00<?, ?it/s]

In [4]:
# Entity.objects.all().delete()

(1825430, {'searchapp.Entity': 1825430})